#  Data Analysis Tools JWebbinar: Specutils

![Specutils: An Astropy Package for Spectroscopy](specutils_logo.png)


This notebook provides an overview of the Astropy coordinated package `specutils`.  While this notebook is intended as an interactive introduction to specutils at the time of its writing, the canonical source of information for the package is the latest version's documentation: 

https://specutils.readthedocs.io

`specutils` should already be in your JWebbinar environment.  If you wish to install locally, you can follow the instructions in [the installation section of the specutils docs](https://specutils.readthedocs.io/en/latest/installation.html).

Once specutils is installed, fundamental imports necessary for this notebook are possible:

# Background/Spectroscopic ecosystem

The large-scale plan for spectroscopy support in the Astropy project is outlined in  the [Astropy Proposal For Enhancement 13](https://github.com/astropy/astropy-APEs/blob/main/APE13.rst).  In summary, this APE13 lays out three broad packages:

* `specutils` - a Python package containing the basic data structures for representing spectroscopic data sets, as well as a suite of fundamental spectroscopic analysis tools to work with these data structures.
* `specreduce` - a general Python package to reduce raw astronomical spectral images to 1d spectra (represented as `specutils` objects).
* `specviz` - a Python package (or possibly suite of packages) for visualization of astronomical spectra.


While all are still in development, the first of these is furthest along, and is the subject of this notebook, as it contains the core data structures and concepts required for the others.

# Fundamentals of specutils

## Objects for representing spectra

The most fundamental purpose of `specutils` is to contain the shared Python-level data structures for storing astronomical spectra.  It is important to recognize that this is not the same as the *on-disk* representation.  As desecribed later specutils provides loaders and writers for various on-disk representations, with the intent that they all load to a common set of in-memory/Python interfaces.  Those intefaces (implemented as Python classes) are described in detail in the [relevant section of the documentation](https://specutils.readthedocs.io/en/latest/types_of_spectra.html), which contains this diagram:

![Specutils Classes](specutils_classes_diagrams.png)

The core principal is that all of these representations contain a `spectral_axis` attribute as well as a `flux` attribute (as well as optional matching `uncertainty`).  The former is often wavelength for OIR spectra, but might be frequency or energy for e.g. Radio or X-ray spectra.  Regardless of which spectral axis is used, the class attempts to interpret it appropriately, using the features of `astropy.Quantity` to distinguish different types of axes.  Similarly, `flux` may or may not be a traditional astronomical `flux` unit (e.g. Jy or  erg sec$^{-1}$ cm$^{-2}$ angstrom$^{-1}$), but is treated as the portion of the spectrum that acts in that manner.  The various classes are then distinguished by whether these attributes are one-dimensional or not, and how to map the `spectral_axis` dimensionality onto the `flux`.  The simplest case (and the one primarily considered here) is the scalar `Spectrum1D` case, which is a single spectrum with a matched-size `flux` and `spectral_axis`.

## Basics of creating Spectrum1D Objects

If your spectrum is in a format that specutils understands, loading it is very straightforward.  There are times when you want a bit more control, though, so lets look at loading from a file and creating an object directly from arrays:

## Loading spectra from files

Specutils comes with readers for a variety of spectral data formats (including loaders for future JWST instruments). While support for specific formats depends primarily from users (like you!) providing readers, you may find that one has already been implemented for your favorite spectrum format.  As an example, we consider a simulated high-redshift (z > 1) galaxy like that you might see from NIRSpec:

# Creating a Spectrum "by-hand"

If you have a format that is not compatible, or you want to do some sort of customization of the loading process, spectra can be created directly from aastropy quantities (which are basically arrays with associated units). Here we'll show how you can do that, using data from the same file above: 

### Exercise

If you have your own spectroscopic data, try loading a file here using either one of the built-in loaders, or the `Spectrum1D` interface, and plotting it.  If you don't have your own data on-hand, try downloading something of interest via a public archive (e.g., public HST data using MAST, or [an sdss galaxy](https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12)), and load it.

If you have time and enough knowledge of the format, try loading your spectrum *both* ways - with a built-in loader and manually creating a `Spectrum1D`.

## Working with Units and Spectral Axes 

We created `Spectrum1D` just as Quantity arrays, so they can be treated just as `Quantity` objects when convenient with unit conversions and the like:

There's even fast-accessors to make some of this more convenient:

But under the hood this are are fully-featured WCS following the [Astropy APE14](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst) WCS interface along with the [GWCS](https://gwcs.readthedocs.io/) package. So you can use that to do conversions to and from spectral to pixel axes:

The other dimension is the flux - that requires slightly more complex transformation because it matters where in the spectrum you are to do the flux transformation:

But specutils makes this much easier!

## Uncertainties

Currently the most compatible way to use uncertainties is the machinery built for the `astropy.nddata` object (although in some cases simply passing in an uncertainty array will also work):

# Arithmetic on Spectra

Specutils provides a lot of functionality for manipulating spectra.  In general these follow the pattern of creating *new* specutils objects with the results of the operation instead of in-place operations.

The most straightforward of operations are arithmetic manipulations.  In general these follow patterns that are based on fundametal arithmetic. E.g.:

However, when there is ambiguity in your intent - for example, two spectra with different units where it is not clear what the desired output is - errors are generally produced instead of the code attempting to guess:

Resolving this requires explicit conversion:

# Sample Analysis: Line Center for Redshift

Specutils has a large set of analysis functions, more than we have time to cover here.  So instead we focus on a specific concrete science case: determining the center of a line and using that to get a redshift estimate.

If you are used to looking at galaxies, you probably recognized that our example spectrum has a characteristic emission line pattern with a strong H$\alpha$ line. While we might be able to center up on a line that strong, to be sure we should always subtract the continuum first.  Many other `specutils` analaysis function expect continuum-subtracted spectra as well.

While there are techniques for identifying the line automatically (see the fitting section below), here we assume we are doing "quick-look" procedures where manual identification is possible. 

We do this by defining a `SpectralRegion` object spanning the area of the line.  Further below is a worked example of how to get these values easily in a notebook, but for now we can take the numbers I've pre-eyeballed: $1.638 \mu m$ to $1.644 \mu m$

You can now call a variety of analysis functions on the continuum-subtracted spectrum to estimate various properties of the line (you can see the full list of relevant analysis functions [in the analysis part of the specutils docs](https://specutils.readthedocs.io/en/stable/analysis.html#functions)). Here we do just the center:

Now we simply plug this into the redshift formula for H$\alpha$'s rest wavelength:

We now have strong evidence this is a manufactured spectrum! (it's suspiciously close to *exactly* 1.5 ...)

### Exercise

Look for other star forming galaxy emission lines. Get their center wavelengths as well, and use those multiple lines to improve your redshift estimate (and perhaps an uncertainty on your estimate).